Setup & Keys

In [1]:
# Cell 1: Setup
from dotenv import load_dotenv
import os

load_dotenv()  # lädt .env

BASE_URL = "https://api.cerebras.ai/v1"
LLM_MODEL = "gpt-oss-120b"
LLM_TEMPERATURE = 0.3
LLM_API_KEY = os.environ["CEREBRAS_API_KEY"]


LLM

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url=BASE_URL,
    api_key=LLM_API_KEY,
    model=LLM_MODEL,
    temperature=LLM_TEMPERATURE,
)

Dokumente laden (gezielte URLs + User-Agent)

In [4]:
# Cell 3: Dokumente laden (PDF, HTML, MD, TXT)
from pathlib import Path
import re
from bs4 import BeautifulSoup
from langchain_community.document_loaders import PyPDFLoader, BSHTMLLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

# --- Einstellungen ---
DATA_DIR = Path("../data").resolve()
RAW_DIR = DATA_DIR / "raw"
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

# --- Text säubern ---
def clean_text(t: str) -> str:
    t = t.replace("\r", "")
    t = re.sub(r"-\n(\w)", r"\1", t)      # Silbentrennungen entfernen
    t = re.sub(r"\n{2,}", "\n\n", t)      # doppelte Zeilenumbrüche
    t = re.sub(r"[ \t]{2,}", " ", t)      # doppelte Leerzeichen
    return t.strip()

# --- Loader-Funktion ---
def load_docs_from_raw():
    assert RAW_DIR.exists(), f"Ordner fehlt: {RAW_DIR.resolve()}"
    docs = []

    for path in RAW_DIR.glob("*"):
        ext = path.suffix.lower()
        if ext == ".pdf":
            try:
                loader = PyPDFLoader(str(path))
                pages = loader.load()
            except Exception:
                print(f"⚠️ Fallback für PDF: {path.name}")
                continue
            for d in pages:
                d.metadata["source"] = path.name
                d.page_content = clean_text(d.page_content)
                docs.append(d)

        elif ext in (".html", ".htm"):
            try:
                loader = BSHTMLLoader(str(path), open_encoding="utf-8", bs_kwargs={"features": "lxml"})
                page_docs = loader.load()
            except Exception:
                # Fallback – manuell mit BeautifulSoup
                raw = path.read_bytes()
                soup = BeautifulSoup(raw, "html.parser")
                text = soup.get_text("\n")
                page_docs = [Document(page_content=text, metadata={"source": path.name})]
            for d in page_docs:
                d.page_content = clean_text(d.page_content)
                docs.append(d)

        elif ext in (".md", ".txt"):
            loader = TextLoader(str(path), encoding="utf-8")
            page_docs = loader.load()
            for d in page_docs:
                d.metadata["source"] = path.name
                d.page_content = clean_text(d.page_content)
                docs.append(d)

        else:
            print(f"⚠️ Überspringe unbekanntes Format: {path.name}")

    print(f"✅ {len(docs)} Dokumente geladen.")
    return docs


# --- Split + Embeddings ---
def build_chunks(docs, chunk_size=800, overlap=120):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    return splitter.split_documents(docs)

def build_faiss(chunks):
    embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)
    return FAISS.from_documents(chunks, embedding=embeddings)

# --- Lade alles ---
docs = load_docs_from_raw()
print("🔹 Beispieltext:", docs[0].page_content[:300])
splits = build_chunks(docs)
vs = build_faiss(splits)
print("FAISS Index erstellt mit:", vs.index.ntotal, "Vektoren")


✅ 3 Dokumente geladen.
🔹 Beispieltext: Garantie und Versicherung | MediaMarkt

Zum Hauptinhalt wechselnAlle KategorienWas suchst du?Mein MarktKein Markt ausgewähltlanguageSwitch.voiceOverDescription: DeutschdeMenüAngebotemyMediaMarktServiceGeschenkkarteMediaMagazinEigenmarkenMarkenshops% Outlet %Shopping CardSmartbarJobsHilfeAlle Kategor
FAISS Index erstellt mit: 54 Vektoren


Splitten

In [5]:
# Cell 4: Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=120)
splits = splitter.split_documents(docs)

len(splits), len(docs), sum(len(s.page_content) for s in splits) // max(1, len(splits))


(73, 3, 543)

Embeddings + FAISS

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# FAISS direkt aus Dokumenten bauen
vs = FAISS.from_documents(splits, embedding=embeddings)

print("FAISS Vektoren:", vs.index.ntotal)


FAISS Vektoren: 73


Prompt/Safeguard

In [7]:

import os
from dotenv import load_dotenv
load_dotenv()


from langchain_core.prompts import ChatPromptTemplate

SAFE_SYSTEM_PROMPT = """
Rolle & Aufgabe:
Du bist eine freundliche und sachliche Kundenberaterin des MediaMarkt Onlineshops (Schweiz).
Deine Aufgabe ist es, Kund:innen bei Fragen zu Produkten, Bestellungen, Rückgaben und allgemeinen Website-Themen zu helfen.

Verhaltensregeln (sehr wichtig):
1. Antworte ausschliesslich auf Basis der unten angegebenen Wissensquellen (Kontext).
2. Wenn im Kontext keine passende Information steht, sag höflich:
   "Dazu habe ich leider keine Informationen. Bitte wende dich direkt an den MediaMarkt Kundendienst."
3. Erfinde oder ergänze keine eigenen Fakten. Keine Spekulationen.
4. Bei rechtlichen oder Garantie-Themen zitiere nur Textstellen aus dem Kontext.
5. Antworte kurz, klar und freundlich auf Deutsch (Schweiz), maximal 3–5 Sätze.
6. Verwende neutrale, respektvolle Sprache und duze die Kund:innen konsequent.
7. Liste am Ende höchstens drei relevante Quellen unter der Überschrift "Quellen:" auf.

Kontextinformationen:
{context}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SAFE_SYSTEM_PROMPT),
    ("human", "{question}")
])


Retriever + Antwort

In [8]:
def _retrieve(vs, query: str, k: int = 6):
    retriever = vs.as_retriever(
        search_type="mmr",
        search_kwargs={"k": k, "fetch_k": 24, "lambda_mult": 0.5},
    )
    # LC 0.1.x hat get_relevant_documents; LC 0.2.x nutzt invoke()
    try:
        return retriever.get_relevant_documents(query)  # 0.1.x
    except AttributeError:
        return retriever.invoke(query)                  # 0.2.x+

def answer_with_rag(question: str, k: int = 6) -> str:
    docs = _retrieve(vs, question, k=k)
    if not docs:
        return "Dazu habe ich keine gesicherten Infos in den FAQs."

    # Kontext + Quellen bauen
    context = "\n\n".join(d.page_content for d in docs[:k])
    seen, sources = set(), []
    for d in docs:
        src = d.metadata.get("source")
        if src and src not in seen:
            sources.append(src); seen.add(src)
        if len(sources) == 3:
            break

    # Prompt und LLM-Aufruf (achte auf den korrekten Import deiner Version)
    try:
        from langchain_core.prompts import ChatPromptTemplate
    except ModuleNotFoundError:
        from langchain_core.prompts import ChatPromptTemplate


    SAFE_SYSTEM_PROMPT = (
        "Du bist Kundenberater:in für MediaMarkt Onlineshop CH.\n"
        "Antworte NUR auf Basis des Kontexts. Wenn nichts passt, sag das offen.\n"
        "Erfinde nichts; gib keine rechtlichen Bewertungen. Zitiere bei Garantie/AGB knappe Textstellen.\n"
        "Kurz & präzise; am Ende max. 3 Quellen unter 'Quellen:'.\n\n"
        "Kontext:\n{context}"
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", SAFE_SYSTEM_PROMPT),
        ("human", "{question}")
    ])
    msgs = prompt.format_messages(context=context, question=question)
    res = llm.invoke(msgs).content

    if sources:
        res += "\n\nQuellen:\n" + "\n".join(f"- {s}" for s in sources)
    return res


Testfragen

In [9]:
print(answer_with_rag("Wie lange ist die Retourenfrist?"))


Die im bereitgestellten Informationen wird keine konkrete Rückgabefrist angegeben.  

**Quellen:**  
- „Wie retourniere ich ein Produkt?“ (Abschnitt zu Rückgabemöglichkeiten)  
- „Ich habe fälschlicherweise ein defektes Produkt als Retoure per Post versendet…“ (Hinweis zu defekten Artikeln)  

Quellen:
- C:\Users\yanni\PycharmProjects\GENAI_Grupp_G\data\raw\stornieren_&_zurückgeben.html
- C:\Users\yanni\PycharmProjects\GENAI_Grupp_G\data\raw\garantie_und_versicherung _ MediaMarkt.html


In [11]:
print(answer_with_rag("Welche Online-Zahlungsmethoden gibt es?"))

Im bereitgestellten Kontext wird lediglich erwähnt, dass MediaMarkt CH „eine Vielzahl an Zahlungsmöglichkeiten“ anbietet, aber es werden keine konkreten Online‑Zahlungsarten (z. B. Kreditkarte, PayPal, Twint usw.) aufgelistet.

**Quellen:**  
- MediaMarkt‑Service‑Übersicht: „…eine Vielzahl an Zahlungsmöglichkeiten…“ (aus dem Kontext)

Quellen:
- C:\Users\yanni\PycharmProjects\GENAI_Grupp_G\data\raw\informieren_&_kaufen.html
- C:\Users\yanni\PycharmProjects\GENAI_Grupp_G\data\raw\garantie_und_versicherung _ MediaMarkt.html


In [12]:
print(answer_with_rag("Wie lange gilt die Herstellergarantie bei MediaMarkt?"))

Die bereitgestellten Informationen enthalten keine Angabe zur Dauer der Herstellergarantie bei MediaMarkt.  

**Quellen:**  
- „Garantie und Versicherung“ (MediaMarkt) – kein Hinweis auf die Garantiedauer.

Quellen:
- C:\Users\yanni\PycharmProjects\GENAI_Grupp_G\data\raw\garantie_und_versicherung _ MediaMarkt.html
- C:\Users\yanni\PycharmProjects\GENAI_Grupp_G\data\raw\stornieren_&_zurückgeben.html
- C:\Users\yanni\PycharmProjects\GENAI_Grupp_G\data\raw\informieren_&_kaufen.html


In [76]:
print(answer_with_rag("Wie retourniere ich ein Produkt?"))

**Rückgabe eines Produkts**

1. **Vor‑Ort‑Retoure im MediaMarkt (empfohlen)**  
   - Besonders nötig, wenn die Originalverpackung bereits geöffnet wurde.  
   - Gehe mit dem Produkt und dem Kaufbeleg zum Kundenservice des nächsten MediaMarkt‑Marktes.  

2. **Rücksendung per Post**  
   - Für ungeöffnete, originalverpackte Artikel kannst du das Produkt per Post an uns zurücksenden.  
   - Nutze das Rücksendeformular aus deiner Bestellbestätigung und lege das Paket gut verpackt bei.  

3. **Abholung für Reparatur (falls nötig)**  
   - Bei defekten Geräten kann das Gerät kostenlos bei dir zuhause abgeholt, repariert und zurückgebracht werden. Ohne Garantieverlängerung kostet die Ab‑ und Zustellung CHF 158.  

**Wichtig:** Das Produkt muss sich in einwandfreiem Zustand (ungeöffnet oder unbenutzt) befinden und die Originalverpackung enthalten, sonst ist die Vor‑Ort‑Retoure zwingend.  

**Quellen**  
- „Bei der Rückgabe eines Produkts stehen dir folgende Möglichkeiten zur Verfügung: a) Vor‑

In [77]:
print(answer_with_rag("Wie werde ich Geschäftskunde?"))

Um Geschäftskunde bei MediaMarkt CH zu werden, nutzen Sie unser B2B‑Portal. Dort finden Sie „massgeschneiderte Lösungen und zahlreiche Vorteile für B2B“ und können ein Kundenkonto anlegen bzw. Kontakt zu unserem Vertrieb aufnehmen. Alle Details und das Anmeldeformular finden Sie unter dem Hinweis **„Alle Infos findest du hier“** im Bereich „Wie werde ich Geschäftskunde?“.

**Quellen**  
- MediaMarkt CH – „Wie werde ich Geschäftskunde? Wir bieten massgeschneiderte Lösungen und zahlreiche Vorteile für B2B an. Alle Infos findest du hier.“

Quellen:
- C:\Users\simon\OneDrive\FH\BAI\GENAI_Grupp_G\data\raw\informieren_&_kaufen.html
- C:\Users\simon\OneDrive\FH\BAI\GENAI_Grupp_G\data\raw\garantie_und_versicherung _ MediaMarkt.html
- zahlungsarten _ mediamarkt.pdf


In [78]:
print(answer_with_rag("Ist Interdiscount besser als Mediamarkt?"))

Ich habe im bereitgestellten Kontext keine Informationen zu Interdiscount, daher kann ich keinen Vergleich zu MediaMarkt ziehen.

**Quellen:**  
- (Keine relevanten Quellen im Kontext)

Quellen:
- C:\Users\simon\OneDrive\FH\BAI\GENAI_Grupp_G\data\raw\garantie_und_versicherung _ MediaMarkt.html
- C:\Users\simon\OneDrive\FH\BAI\GENAI_Grupp_G\data\raw\Zahlungsarten _ MediaMarkt.html
- C:\Users\simon\OneDrive\FH\BAI\GENAI_Grupp_G\data\raw\informieren_&_kaufen.html


In [16]:
import gradio as gr

# --- Gradio UI ---
def chat_interface(user_question):
    # Ruft euren bestehenden RAG-Agenten auf:
    response = answer_with_rag(user_question)
    return response

# --- Custom CSS für MediaMarkt Style ---
custom_css = """
body {
    background-color: #ffffff;
    font-family: 'Helvetica', sans-serif;
}
.gradio-container {
    border: 3px solid #e60000;
    border-radius: 20px;
    padding: 25px;
    background: linear-gradient(180deg, #ffffff 0%, #ffecec 100%);
    box-shadow: 0 0 25px rgba(230, 0, 0, 0.1);
}
h1, h2, h3 {
    color: #e60000;
    text-align: center;
    font-weight: 900;
    text-transform: uppercase;
}
textarea {
    border: 2px solid #e60000 !important;
    border-radius: 12px !important;
    padding: 12px !important;
    background-color: #fffafa !important;
    font-size: 16px !important;
}
button {
    background-color: #e60000 !important;
    color: #ffffff !important;
    border-radius: 12px !important;
    border: none !important;
    font-weight: bold !important;
    transition: all 0.3s ease-in-out !important;
}
button:hover {
    background-color: #b30000 !important;
    transform: scale(1.05);
}
.output-markdown {
    background-color: #fffafa !important;
    border: 1px solid #e60000 !important;
    border-radius: 12px !important;
    padding: 15px !important;
}
footer {
    text-align: center;
    font-size: 13px;
    color: #666;
    margin-top: 15px;
}
"""

demo = gr.Interface(
    fn=chat_interface,
    inputs=gr.Textbox(
        label="💬 Frage:",
        placeholder="Frag den MediaMarkt Kundenservice...",
        lines=2,
    ),
    outputs=gr.Markdown(label="📖 Antwort"),
    title="🛒 MediaMarkt KI-Kundenservice",
    description="Dieser KI-Agent beantwortet Fragen auf Basis echter MediaMarkt-Dokumente (RAG + Safeguarding).",
    theme="gradio/soft",
    examples=[
        ["Wie lange ist die Retourenfrist?"],
        ["Wie reklamiere ich ein defektes Produkt?"],
        ["Welche Zahlungsmöglichkeiten gibt es?"],
    ],
    css=custom_css,  # <- MediaMarkt Design anwenden
)

if __name__ == "__main__":
    demo.launch(share=False)


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
